In [1]:
from __future__ import print_function, division
# from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
import torchvision.models as models
# import matplotlib.pyplot as plt
import torch.optim as optim
from PIL import Image
import numpy as np
import random
import torch
import copy
import time
import os

from image_loader import SchoolDataset
from attnNet import *
from helpers import *

In [2]:
dataset = SchoolDataset("./data/y1314_AllSubjects.csv", "./data/imagery/")

x_train, y_train, x_val, y_val, w_train, w_val = train_test_split_regr(dataset, .80)

train = [(k,v, w) for k,v,w in zip(x_train, y_train, w_train)]
val = [(k,v, w) for k,v,w in zip(x_val, y_val, w_val)]


print(len(train))
print(len(val))

dataset_sizes = {}
dataset_sizes['train'] = len(train)
dataset_sizes['val'] = len(val)

batchSize = 32

# Prep the training and validation set
train = torch.utils.data.DataLoader(train, batch_size = batchSize, shuffle = True)
val = torch.utils.data.DataLoader(val, batch_size = batchSize, shuffle = True)


dataloaders = {}
dataloaders['train'] = train
dataloaders['val'] = val

4700
1175


In [3]:
def train_regr_model(model, optimizer, criterion, dataloaders, dataset_sizes, device, batch_size, num_epochs=25):

    epoch_num = 0

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_mae = 9000000000000000000

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                # Set model to training mode
                model.train() 
            else:
              # Set model to evaluate mode
                model.eval()  

            running_loss = 0.0
            running_mae = 0

            # Iterate over data.
            for inputs, labels, weights in dataloaders[phase]:

                inputs = inputs.to(device)
                labels = labels.to(device)
                weights = weights.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward - track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)

                    # print(outputs)
                    # loss = criterion(outputs, labels.view(-1, 1))
                    loss = weighted_loss(outputs, labels.view(-1, 1), weights.view(-1, 1))

                    # print(labels.view(-1, 1))
                    # print(outputs)

                    # backward + optimize only if in training phase
                    if phase == 'train':

                        # for n,p in model.named_parameters():
                        #     print(p)

                        loss.backward()
                        optimizer.step()

                        # asdga

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_mae += mae(outputs, labels.view(-1, 1)).item()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_mae = running_mae / dataset_sizes[phase]

            print('{} Loss: {:.4f} MAE: {:.4f}'.format(
                phase, epoch_loss, epoch_mae))

            # deep copy the model
            if phase == 'val' and epoch_mae < best_mae:
                best_mae = epoch_mae
                best_model_wts = copy.deepcopy(model.state_dict())
                print("Updating model weights.")

        epoch_num += 1



    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_mae))
    print("\n")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
import numpy as np
import random
import torch



class channelMaxPool(torch.nn.Module):
    def __init__(self, in_channels, h, w, batch_size):
        super(channelMaxPool, self).__init__()
        self.batch_size = batch_size
        self.h = h
        self.w = h


    def forward(self, x):
        # torch.reshape(v, (1,2,2))
        x, i = torch.max(x, dim = 1)
        return torch.reshape(x, (x.shape[0], 1, self.h, self.w))


class spatialMaxPool(torch.nn.Module):
    def __init__(self, in_channels, batch_size):
        super(spatialMaxPool, self).__init__()
        self.in_channels = in_channels
        self.batch_size = batch_size

    def forward(self, x):
        x, i = torch.max(x, dim = -1)
        x, i = torch.max(x, dim = -1)
        return torch.reshape(x, (x.shape[0], self.in_channels, 1, 1))


class channelAvgPool(torch.nn.Module):
    def __init__(self, in_channels, h, w, batch_size):
        super(channelAvgPool, self).__init__()
        self.batch_size = batch_size
        self.h = h
        self.w = h


    def forward(self, x):
        x = torch.mean(x, dim = 1)
        return torch.reshape(x, (x.shape[0], 1, self.h, self.w))


class spatialAvgPool(torch.nn.Module):
    def __init__(self, in_channels, batch_size):
        super(spatialAvgPool, self).__init__()
        self.in_channels = in_channels
        self.batch_size = batch_size

    def forward(self, x):
        x = torch.mean(x, dim = -1)
        x = torch.mean(x, dim = -1)
        return torch.reshape(x, (x.shape[0], self.in_channels, 1, 1))



class attnNet(torch.nn.Module):
    def __init__(self, in_channels, h, w, batch_size, resnet):
        super(attnNet, self).__init__()

        # Normal resnet stuff
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        self.linear = torch.nn.Linear(in_features=2048, out_features=1, bias = True)


        # Attention layers
        self.sMP = spatialMaxPool(in_channels = 2048, batch_size = batch_size)
        self.cMP = channelMaxPool(in_channels = 2048, h = 7, w = 7, batch_size = batch_size)
        self.sAP = spatialAvgPool(in_channels = 2048, batch_size = batch_size)
        self.cAP = channelAvgPool(in_channels = 2048, h = 7, w = 7, batch_size = batch_size)
        # self.out_channels = int(in_channels/16)
        self.out_channels = in_channels
        self.convR_M = torch.nn.Conv2d(in_channels = 2048, out_channels = self.out_channels, kernel_size = (1,1), bias=True)
        self.convA_M = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (1,1), bias=True)
        self.convB_M = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (3,3), bias=True, padding = 1)
        self.convC_M = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (7,7), bias=True, padding = 3)
        self.convE_M = torch.nn.Conv2d(in_channels = self.out_channels * 3, out_channels = 2048, kernel_size = (1,1), bias=True)
        
        self.convR_A = torch.nn.Conv2d(in_channels = 2048, out_channels = self.out_channels, kernel_size = (1,1), bias=True)
        self.convA_A = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (1,1), bias=True)
        self.convB_A = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (3,3), bias=True, padding = 1)
        self.convC_A = torch.nn.Conv2d(in_channels = self.out_channels, out_channels = self.out_channels, kernel_size = (7,7), bias=True, padding = 3)
        self.convE_A = torch.nn.Conv2d(in_channels = self.out_channels * 3, out_channels = 2048, kernel_size = (1,1), bias=True)

        self.bn2 = torch.nn.BatchNorm2d(2048)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):

        # print(x.shape)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # out = self.avgpool(out)
        # out = out.flatten(start_dim=1)
        # out = self.linear(out)

        # print(out.shape)

        out = self.bn2(out)


        # Max Pooling
        fsM = self.sMP(out)
        fcM = self.cMP(out)
        fscM = torch.mul(fsM, fcM)
        rM = self.convR_M(fscM)
        aM = self.convA_M(rM)
        bM = self.convB_M(rM)
        cM = self.convC_M(rM)
        catM = torch.cat((aM,bM,cM), dim = 1)
        eM = self.convE_M(catM)


        # Avg Pooling
        fsA = self.sAP(out)
        fcA = self.cAP(out)
        fscA = torch.mul(fsA, fcA)
        rA = self.convR_A(fscA)
        aA = self.convA_A(rA)
        bA = self.convB_A(rA)
        cA = self.convC_A(rA)
        catA = torch.cat((aA,bA,cA), dim = 1)
        eA = self.convE_A(catA)

        added = torch.add(eA, eM)
        # print(added)

        # print(self.relu(added))

        # added_norm = self.bn2(added)

        attn_mask = self.relu(added)

        out = torch.add(out, attn_mask)

        # print("A: ", out)

        out = self.avgpool(out)

        # print("B: ", out.shape)

        out = out.flatten(start_dim=1)
        out = self.linear(out)

        return out

In [9]:
resnet50 = models.resnet50()
IN_CHANNELS = 3
H = 224
W = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = attnNet(IN_CHANNELS, H, W, batchSize, resnet50)
model = model.to(device)
criterion = torch.nn.L1Loss(reduction = 'mean')
optimizer = optim.Adam(model.parameters(), lr=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [10]:
model = train_regr_model(model, optimizer, criterion, dataloaders, dataset_sizes, device, batchSize, num_epochs = 25)

Epoch 0/24
----------
train Loss: 0.8497 MAE: 1.2494
val Loss: 0.7533 MAE: 0.8418
Updating model weights.
Epoch 1/24
----------
train Loss: 0.6496 MAE: 0.9224
val Loss: 0.6557 MAE: 0.9965
Epoch 2/24
----------
train Loss: 0.6376 MAE: 0.9200
val Loss: 0.6519 MAE: 0.8585
Epoch 3/24
----------
train Loss: 0.6230 MAE: 0.8976
val Loss: 0.6710 MAE: 0.8392
Updating model weights.
Epoch 4/24
----------
train Loss: 0.6043 MAE: 0.8733
val Loss: 0.6805 MAE: 0.8418
Epoch 5/24
----------
train Loss: 0.5947 MAE: 0.8649
val Loss: 0.6254 MAE: 0.8650
Epoch 6/24
----------
train Loss: 0.5826 MAE: 0.8472
val Loss: 0.7737 MAE: 0.8877
Epoch 7/24
----------
train Loss: 0.5522 MAE: 0.8174
val Loss: 0.8096 MAE: 0.9375
Epoch 8/24
----------
train Loss: 0.5272 MAE: 0.7924
val Loss: 0.6701 MAE: 1.0980
Epoch 9/24
----------
train Loss: 0.5121 MAE: 0.7858
val Loss: 0.7093 MAE: 0.9868
Epoch 10/24
----------
train Loss: 0.5008 MAE: 0.7588
val Loss: 0.7293 MAE: 1.2591
Epoch 11/24
----------
train Loss: 0.4682 MAE: 0.

In [11]:
torch.save({
            'epoch': 25,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
        }, "./trained_models/schoolCNN_L0001_r50_wl_b32_bn_v1.torch")